In [1]:
!unzip /content/drive/MyDrive/open.zip

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
  inflating: upscale_train/TRAIN_10834.png  
  inflating: upscale_train/TRAIN_10835.png  
  inflating: upscale_train/TRAIN_10836.png  
  inflating: upscale_train/TRAIN_10837.png  
  inflating: upscale_train/TRAIN_10838.png  
  inflating: upscale_train/TRAIN_10839.png  
  inflating: upscale_train/TRAIN_10840.png  
  inflating: upscale_train/TRAIN_10841.png  
  inflating: upscale_train/TRAIN_10842.png  
  inflating: upscale_train/TRAIN_10843.png  
  inflating: upscale_train/TRAIN_10844.png  
  inflating: upscale_train/TRAIN_10845.png  
  inflating: upscale_train/TRAIN_10846.png  
  inflating: upscale_train/TRAIN_10847.png  
  inflating: upscale_train/TRAIN_10848.png  
  inflating: upscale_train/TRAIN_10849.png  
  inflating: upscale_train/TRAIN_10850.png  
  inflating: upscale_train/TRAIN_10851.png  
  inflating: upscale_train/TRAIN_10852.png  
  inflating: upscale_train/TRAIN_10853.png  
  inflating: upscale_train/TRAIN_10854.png  
  inflating: upscal

In [2]:
import pandas as pd
train_df = pd.read_csv('/content/train.csv')
test_df = pd.read_csv('/content/test.csv')

In [3]:
train_df.head()

,img_path,upscale_img_path,label
0,./train/TRAIN_00000.jpg,./upscale_train/TRAIN_00000.png,Ruddy Shelduck
1,./train/TRAIN_00001.jpg,./upscale_train/TRAIN_00001.png,Gray Wagtail
2,./train/TRAIN_00002.jpg,./upscale_train/TRAIN_00002.png,Indian Peacock
3,./train/TRAIN_00003.jpg,./upscale_train/TRAIN_00003.png,Common Kingfisher
4,./train/TRAIN_00004.jpg,./upscale_train/TRAIN_00004.png,Common Kingfisher


In [4]:
test_df.head()

,id,img_path
0,TEST_00000,./test/TEST_00000.jpg
1,TEST_00001,./test/TEST_00001.jpg
2,TEST_00002,./test/TEST_00002.jpg
3,TEST_00003,./test/TEST_00003.jpg
4,TEST_00004,./test/TEST_00004.jpg


In [5]:
submission_df = pd.read_csv('/content/sample_submission.csv')
submission_df.head()

,id,label
0,TEST_00000,Indian Roller
1,TEST_00001,Indian Roller
2,TEST_00002,Indian Roller
3,TEST_00003,Indian Roller
4,TEST_00004,Indian Roller


In [6]:
# 필요 라이브러리
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten,Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.resnet50 import ResNet50,preprocess_input

In [7]:
train_df.label.value_counts()

label
Ruddy Shelduck               671
White-Breasted Kingfisher    666
Red-Wattled Lapwing          663
Northern Lapwing             663
Gray Wagtail                 661
White Wagtail                659
Jungle Babbler               658
Sarus Crane                  657
Common Kingfisher            657
Common Rosefinch             653
White-Breasted Waterhen      649
Indian Grey Hornbill         645
Common Tailorbird            643
Brown-Headed Barbet          642
Cattle Egret                 641
Indian Roller                639
Indian Peacock               637
Hoopoe                       631
House Crow                   630
Common Myna                  630
Forest Wagtail               629
Rufous Treepie               616
Asian Green Bee-Eater        613
Coppersmith Barbet           550
Indian Pitta                 431
Name: count, dtype: int64

In [8]:
len(np.unique(train_df.label))

25

In [9]:
# 이미지 데이터를 수치화
from PIL import Image
from glob import glob
# img = Image.open(files_list[0])
# np.array(img.resize( (32,32) )) / 255.0
X_train = []
for file in train_df.img_path:
  img = Image.open(file)
  X_train.append(np.array(img.resize( (64,64) )) / 255.0)

In [10]:
X = np.array(X_train)
train_df['source'] = X

In [11]:
y = train_df.label.to_numpy()
y.shape

(15834,)

In [15]:
from sklearn.preprocessing import LabelEncoder
y = LabelEncoder().fit_transform(y)
y

array([19,  9, 13, ...,  2, 19,  8])

In [16]:
# 데이터를학습용과 검증용으로 분리
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y ,stratify=y, random_state=42)

In [ ]:
base_model =  ResNet50(include_top=False,input_shape=(64,64,3))
# 분류기 없는 convolution 층만 구성되어 있다
cnn = Sequential()
cnn.add(base_model)
cnn.add(Flatten())  # 1차원 데이터형태로 변경
cnn.add(Dense(1024,activation='relu')  )
cnn.add(Dense(25,activation = 'softmax'))   # no_class 200 다중분류 문제

In [ ]:
ealy_stopping = tf.keras.callbacks.EarlyStopping(patience=2, restore_best_weights=True)
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint('/content/drive/MyDrive/dacon_bird_resnet50.keras', save_best_only=True)

cnn.compile(optimizer=Adam(learning_rate=1e-5),
            loss='sparse_categorical_crossentropy', metrics=['accuracy'] )
hist = cnn.fit(X_train,y_train,epochs=5,validation_data=(X_test, y_test), callbacks=[ealy_stopping, checkpoint_cb])

Epoch 1/5
